In [18]:
import cv2
import os
from PIL import Image
import face_recognition
from tqdm import tqdm_notebook
import pytube
from fastai.vision.all import *
from deepface import DeepFace
import time
import torchaudio, torch
import moviepy.editor as mp
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [19]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [20]:
cd C:\Users\Adnan\Desktop\Data Science\Final Capstone Project

C:\Users\Adnan\Desktop\Data Science\Final Capstone Project


In [21]:
version = 3
model_path = f'models/gaze-recognizer-v{version}.pkl'

In [22]:
model = load_learner(model_path)

In [23]:
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe'
]

In [25]:
angry = 0
disgust = 0
fear = 0
happy = 0
sad = 0
surprise = 0
neutral = 0
emotion_count = 0
# Open the video file
video_source = 'source_videos/A message from longbeachgriffy.mp4'
video_capture = cv2.VideoCapture(video_source)
transcription = getTranscription(video_source)
print(transcription)

# Get the total number of frames in the video
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)
start = time.time()
# Loop through the frames of the video
# while True:
on_camera = 0
off_camera = 0
total = 0
for _ in tqdm_notebook(range(total_frames)):
    # Read a single frame from the video
    for i in range(24*5):
        ret, frame = video_capture.read()
        if not ret:
            break

    # If there are no more frames, break out of the loop
    if not ret:
        break
    
    # Convert the frame to RGB color (face_recognition uses RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    

    # Find all the faces in the frame using a pre-trained convolutional neural network.
    face_locations = face_recognition.face_locations(gray)
    #face_locations = face_recognition.face_locations(gray, number_of_times_to_upsample=0, model="cnn")

    if len(face_locations) > 0:
        # Show the original frame with face rectangles drawn around the faces
        for top, right, bottom, left in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            face_image = gray[top:bottom, left:right]
            color_image = frame[top:bottom, left:right]
            #plt.imshow(face_image[:,::-1])

            # Resize the face image to the desired size
            resized_face_image = cv2.resize(face_image, (128,128))
            
            try:
                emotion = DeepFace.analyze(color_image,actions=['emotion'],detector_backend = backends[2],enforce_detection = False)# 2,3, 4 works
                emotion_count += 1
                print(emotion[0]['emotion'])
                angry += emotion[0]['emotion']['angry']
                disgust += emotion[0]['emotion']['disgust']
                fear += emotion[0]['emotion']['fear']
                happy += emotion[0]['emotion']['happy']
                sad += emotion[0]['emotion']['sad']
                surprise += emotion[0]['emotion']['surprise']
                neutral += emotion[0]['emotion']['neutral']
            except Exception as e:
                pass
            

            #emotions = emotions+emotion.emotion
            # 4 took 110 3 took 58 2 took 57

            # Predict the class of the resized face image using the model
            result = model.predict(resized_face_image)
            print(result[0])
            if(result[0] == 'on_camera'): on_camera = on_camera + 1
            elif(result[0] == 'off_camera'): off_camera = off_camera + 1
            total = total + 1
        
    cv2.imshow('Video', frame)

    # If the user presses the 'q' key, exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# print(total,on_camera,off_camera)
print(f'focus perfectage = {on_camera/total*100}')
# Release the video capture object and close all windows
video_capture.release()
cv2.destroyAllWindows()
angry = angry / emotion_count
disgust = disgust / emotion_count
fear = fear / emotion_count
happy = happy / emotion_count
sad = sad / emotion_count
surprise = surprise / emotion_count
neutral = neutral / emotion_count
print(f'total anger percentage = {angry}')
print(f'total disgust percentage = {disgust}')
print(f'total fear percentage = {fear}')
print(f'total happy percentage = {happy}')
print(f'total sad percentage = {sad}')
print(f'total surprise percentage = {surprise}')
print(f'total neutral percentage = {neutral}')
end = time.time()
print(end-start)


MoviePy - Writing audio in audio.wav


MoviePy - Done.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


 Hey, just want to tell you you're doing good You're doing your absolute best you're trying your absolute hardest and that's what really matters Don't be so hard on yourself You're doing amazing and if nobody told you lately, I'm proud of you
905


  0%|          | 0/905 [00:00<?, ?it/s]

...

{'angry': 0.03949433157686144, 'disgust': 1.4949245183762394e-10, 'fear': 0.0017348404071526602, 'happy': 99.8606026172638, 'sad': 0.014265705249272287, 'surprise': 0.004526855263975449, 'neutral': 0.07938353810459375}


on_camera



Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.25it/s]

{'angry': 0.0014545236808771733, 'disgust': 6.31178594633075e-12, 'fear': 0.01961608068086207, 'happy': 99.95170831680298, 'sad': 1.3689509614778217e-05, 'surprise': 0.011821027146652341, 'neutral': 0.015392844215966761}


on_camera
focus perfectage = 100.0
total anger percentage = 2.241548837287155
total disgust percentage = 0.0005319615337162579
total fear percentage = 0.3972631010583427
total happy percentage = 76.80924758315086
total sad percentage = 19.19191134706466
total surprise percentage = 0.006083648120380758
total neutral percentage = 1.3534165373130236
7.6690661907196045


In [16]:
def getTranscription(path):
    # Insert Local Video File Path
    clip = mp.VideoFileClip(path)

    # Insert Local Audio File Path
    clip.audio.write_audiofile(r"audio.wav")
    
    waveform, sample_rate = torchaudio.load("audio.wav")
    resampler = torchaudio.transforms.Resample(sample_rate, 16000)
    waveform = resampler(waveform)[0]
    
    processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
    model.config.forced_decoder_ids = None
    
    input_features = processor(waveform.squeeze(dim=0), return_tensors="pt").input_features 
    predicted_ids = model.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    
    return transcription[0]
    